In [5]:
# Standard library imports
import os                          # To interact with the operating system
import shutil                      # For file operations (copy, move, etc.)
from io import BytesIO             # For byte operations

# Numerical and data manipulation libraries
import numpy as np                 # For numerical operations

# Data visualization libraries
import matplotlib.pyplot as plt     # For plotting graphs
import seaborn as sns             # For data visualization

# Image processing libraries
from PIL import Image, ImageOps, ImageFile     # For image processing
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # For loading images

# Audio analysis library
import librosa                     # For audio analysis

# Machine learning libraries
from sklearn.model_selection import train_test_split  # For splitting data into train and test sets
from sklearn.preprocessing import LabelEncoder  # For encoding labels
from sklearn.preprocessing import StandardScaler  # For standardizing data
from sklearn.metrics import confusion_matrix, classification_report  # For evaluation metrics

# TensorFlow/Keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential  # For creating sequential models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # For defining layers
from tensorflow.keras.utils import to_categorical  # For converting labels to categorical format
from tensorflow.keras.optimizers import Adam  # For optimization algorithms
from tensorflow.keras.applications import MobileNet  # For pre-trained MobileNet model
from tensorflow.keras.callbacks import EarlyStopping  # For early stopping during training

In [6]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
data_path = '/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset'

data = []   # List to store image arrays
labels = []  # List to store corresponding labels for each image

# Get a list of categories (subdirectories) in the dataset
categories = os.listdir(data_path)

# Loop through each category (i.e., each subdirectory)
for category in categories:
    category_path = os.path.join(data_path, category)  # Construct the full path to the category directory
    
    # Loop over each image in the current category
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)  # Construct the full path to the image file
        
        # Load and resize the image to (224, 224) pixels
        img = load_img(img_path, target_size=(224, 224))
        
        # Convert the image to a NumPy array
        img_array = img_to_array(img)
        
        # Normalize pixel values to the range [0, 1]
        img_array /= 255.0
        
        # Append the processed image array to the data list
        data.append(img_array)
        
        # Append the corresponding category label to the labels list
        labels.append(category)

# Encode the labels into numerical format
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Convert the encoded labels to a categorical format (one-hot encoding)
categorical_labels = to_categorical(encoded_labels)

In [8]:
label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(labels)

# Convert the encoded labels to a categorical format (one-hot encoding)
categorical_labels = to_categorical(encoded_label)

# Split the dataset into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data, categorical_labels, test_size=0.2, random_state=42)

# Further split the test set into test and validation sets (50% each)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Convert lists to NumPy arrays for easier processing in model training
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Print the shapes of the datasets to confirm their dimensions
print(f'X_train shape is {X_train.shape}')
print(f'X_val shape is {X_val.shape}')
print(f'X_test shape is {X_test.shape}')
print(f'y_train shape is {y_train.shape}')
print(f'y_val shape is {y_val.shape}')
print(f'y_test shape is {y_test.shape}')


X_train shape is (6284, 224, 224, 3)
X_val shape is (786, 224, 224, 3)
X_test shape is (786, 224, 224, 3)
y_train shape is (6284, 31)
y_val shape is (786, 31)
y_test shape is (786, 31)


In [9]:
#  Model building + training (uses your X_train, X_val, X_test, y_train, y_val, y_test) 
import os, json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet  # you imported MobileNet earlier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

# --------- Config ----------
IMG_SIZE = (224, 224)
BATCH = 32            # reduce to 16 or 8 if you get OOM
AUTOTUNE = tf.data.AUTOTUNE
OUT_DIR = "/kaggle/working"
os.makedirs(OUT_DIR, exist_ok=True)
SEED = 42

# --------- Ensure arrays are float32 and labels are float32 as well ----------
X_train = X_train.astype("float32")
X_val   = X_val.astype("float32")
X_test  = X_test.astype("float32")
y_train = y_train.astype("float32")
y_val   = y_val.astype("float32")
y_test  = y_test.astype("float32")

# --------- Build tf.data pipelines from numpy arrays (fast + supports prefetch/AUTOTUNE) ----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(X_train), seed=SEED).batch(BATCH).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH).prefetch(AUTOTUNE)

# --------- Build model: MobileNet backbone (pretrained) + head ----------
NUM_CLASSES = y_train.shape[1]  # should be 31 in your dataset

# load MobileNet (pretrained on ImageNet) without top classifier
base_model = MobileNet(weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False   # STAGE 1: freeze backbone (feature extraction)

# build classification head
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)            # keep base in inference mode
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = models.Model(inputs, outputs)

# compile for stage 1
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# --------- Callbacks ----------
ckpt1 = ModelCheckpoint(os.path.join(OUT_DIR, "best_stage1.h5"),
                        save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)
es1 = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True, verbose=1)
rlp = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

# --------- STAGE 1: train head only (feature extraction) ----------
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,                # enough for head; early stopping will cut if needed
    callbacks=[ckpt1, es1, rlp],
    verbose=1
)

# ensure best weights loaded
if os.path.exists(os.path.join(OUT_DIR, "best_stage1.h5")):
    model.load_weights(os.path.join(OUT_DIR, "best_stage1.h5"))

# --------- STAGE 2: fine-tune top layers of the backbone ----------
# unfreeze backbone
base_model.trainable = True

# freeze lower layers, fine-tune top ~30% (adjustable)
fine_tune_at = int(len(base_model.layers) * 0.7)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# recompile with a much lower LR for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

ckpt2 = ModelCheckpoint(os.path.join(OUT_DIR, "best_finetuned.h5"),
                        save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)
es2 = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,             # small number, fine-tuning doesn't need many epochs
    callbacks=[ckpt2, es2, rlp],
    verbose=1
)

# load best fine-tuned weights
if os.path.exists(os.path.join(OUT_DIR, "best_finetuned.h5")):
    model.load_weights(os.path.join(OUT_DIR, "best_finetuned.h5"))

# --------- Save final model ----------
final_model_path = os.path.join(OUT_DIR, "mobilenet_finetuned.h5")
model.save(final_model_path)
print("Saved model to:", final_model_path)

# --------- Evaluate on test set and show classification report ----------
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"Test loss: {test_loss:.4f}  Test accuracy: {test_acc:.4f}")

# get predictions and printable report
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)   # y_test is numpy array of one-hot

print("\nClassification report (per-class):")
print(classification_report(y_true_labels, y_pred_labels, digits=4))

# Confusion matrix (optional - can be large)
cm = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion matrix shape:", cm.shape)


I0000 00:00:1755905831.673510      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1755905831.674180      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 31)             │        31,775 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,260,639 (12.44 MB)

...

Epoch 1/30


I0000 00:00:1755905853.469331      98 service.cc:148] XLA service 0x7d5ca8126700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755905853.470811      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1755905853.470831      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1755905854.117575      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  7/197 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.0677 - loss: 4.0611

I0000 00:00:1755905859.155789      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1816 - loss: 3.0755
Epoch 1: val_accuracy improved from -inf to 0.62468, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 26s 67ms/step - accuracy: 0.1822 - loss: 3.0726 - val_accuracy: 0.6247 - val_loss: 1.4420 - learning_rate: 0.0010
Epoch 2/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5598 - loss: 1.5062
Epoch 2: val_accuracy improved from 0.62468 to 0.72392, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.5602 - loss: 1.5046 - val_accuracy: 0.7239 - val_loss: 1.0600 - learning_rate: 0.0010
Epoch 3/30
196/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6969 - loss: 1.0570
Epoch 3: val_accuracy improved from 0.72392 to 0.74173, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.6969 - loss: 1.0567 - val_accuracy: 0.7417 - val_loss: 0.9204 - learning_rate: 0.0010
Epoch 4/30
196/19

Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf_no_top.h5
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model: "functional"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 31)             │        31,775 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 3,260,639 (12.44 MB)
 Trainable params: 31,775 (124.12 KB)
 Non-trainable params: 3,228,864 (12.32 MB)
Epoch 1/30
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1755905853.469331      98 service.cc:148] XLA service 0x7d5ca8126700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755905853.470811      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1755905853.470831      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1755905854.117575      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
  7/197 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.0677 - loss: 4.0611
I0000 00:00:1755905859.155789      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1816 - loss: 3.0755
Epoch 1: val_accuracy improved from -inf to 0.62468, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 26s 67ms/step - accuracy: 0.1822 - loss: 3.0726 - val_accuracy: 0.6247 - val_loss: 1.4420 - learning_rate: 0.0010
Epoch 2/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5598 - loss: 1.5062
Epoch 2: val_accuracy improved from 0.62468 to 0.72392, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.5602 - loss: 1.5046 - val_accuracy: 0.7239 - val_loss: 1.0600 - learning_rate: 0.0010
Epoch 3/30
196/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6969 - loss: 1.0570
Epoch 3: val_accuracy improved from 0.72392 to 0.74173, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.6969 - loss: 1.0567 - val_accuracy: 0.7417 - val_loss: 0.9204 - learning_rate: 0.0010
Epoch 4/30
196/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7300 - loss: 0.9018
Epoch 4: val_accuracy improved from 0.74173 to 0.78626, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.7301 - loss: 0.9015 - val_accuracy: 0.7863 - val_loss: 0.7844 - learning_rate: 0.0010
Epoch 5/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7800 - loss: 0.7567
Epoch 5: val_accuracy did not improve from 0.78626
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7799 - loss: 0.7569 - val_accuracy: 0.7583 - val_loss: 0.7820 - learning_rate: 0.0010
Epoch 6/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7845 - loss: 0.6949
Epoch 6: val_accuracy improved from 0.78626 to 0.79135, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.7845 - loss: 0.6949 - val_accuracy: 0.7913 - val_loss: 0.6976 - learning_rate: 0.0010
Epoch 7/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8347 - loss: 0.5960
Epoch 7: val_accuracy improved from 0.79135 to 0.80025, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8344 - loss: 0.5962 - val_accuracy: 0.8003 - val_loss: 0.6897 - learning_rate: 0.0010
Epoch 8/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8404 - loss: 0.5531
Epoch 8: val_accuracy did not improve from 0.80025
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8403 - loss: 0.5533 - val_accuracy: 0.7952 - val_loss: 0.6837 - learning_rate: 0.0010
Epoch 9/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8508 - loss: 0.5275
Epoch 9: val_accuracy did not improve from 0.80025
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8507 - loss: 0.5277 - val_accuracy: 0.7888 - val_loss: 0.6859 - learning_rate: 0.0010
Epoch 10/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8466 - loss: 0.4991
Epoch 10: val_accuracy did not improve from 0.80025
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8466 - loss: 0.4993 - val_accuracy: 0.7952 - val_loss: 0.6543 - learning_rate: 0.0010
Epoch 11/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8673 - loss: 0.4551
Epoch 11: val_accuracy improved from 0.80025 to 0.80789, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8671 - loss: 0.4554 - val_accuracy: 0.8079 - val_loss: 0.6193 - learning_rate: 0.0010
Epoch 12/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8638 - loss: 0.4498
Epoch 12: val_accuracy did not improve from 0.80789
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8638 - loss: 0.4499 - val_accuracy: 0.7964 - val_loss: 0.6384 - learning_rate: 0.0010
Epoch 13/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8762 - loss: 0.4271
Epoch 13: val_accuracy improved from 0.80789 to 0.81552, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8761 - loss: 0.4272 - val_accuracy: 0.8155 - val_loss: 0.6026 - learning_rate: 0.0010
Epoch 14/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8804 - loss: 0.4041
Epoch 14: val_accuracy did not improve from 0.81552
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8804 - loss: 0.4043 - val_accuracy: 0.8092 - val_loss: 0.6212 - learning_rate: 0.0010
Epoch 15/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8813 - loss: 0.3935
Epoch 15: val_accuracy did not improve from 0.81552

Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8812 - loss: 0.3936 - val_accuracy: 0.8104 - val_loss: 0.6162 - learning_rate: 0.0010
Epoch 16/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8921 - loss: 0.3494
Epoch 16: val_accuracy did not improve from 0.81552
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8921 - loss: 0.3495 - val_accuracy: 0.8066 - val_loss: 0.6081 - learning_rate: 5.0000e-04
Epoch 17/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8978 - loss: 0.3674
Epoch 17: val_accuracy improved from 0.81552 to 0.81679, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.8978 - loss: 0.3673 - val_accuracy: 0.8168 - val_loss: 0.5922 - learning_rate: 5.0000e-04
Epoch 18/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8922 - loss: 0.3636
Epoch 18: val_accuracy did not improve from 0.81679
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8923 - loss: 0.3635 - val_accuracy: 0.8053 - val_loss: 0.6142 - learning_rate: 5.0000e-04
Epoch 19/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9007 - loss: 0.3372
Epoch 19: val_accuracy did not improve from 0.81679
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9007 - loss: 0.3374 - val_accuracy: 0.8079 - val_loss: 0.5867 - learning_rate: 5.0000e-04
Epoch 20/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8934 - loss: 0.3535
Epoch 20: val_accuracy improved from 0.81679 to 0.82061, saving model to /kaggle/working/best_stage1.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8933 - loss: 0.3535 - val_accuracy: 0.8206 - val_loss: 0.5795 - learning_rate: 5.0000e-04
Epoch 21/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8999 - loss: 0.3406
Epoch 21: val_accuracy did not improve from 0.82061
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8999 - loss: 0.3405 - val_accuracy: 0.8053 - val_loss: 0.6086 - learning_rate: 5.0000e-04
Epoch 22/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9014 - loss: 0.3226
Epoch 22: val_accuracy did not improve from 0.82061

Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9014 - loss: 0.3227 - val_accuracy: 0.8117 - val_loss: 0.5969 - learning_rate: 5.0000e-04
Epoch 23/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9099 - loss: 0.3157
Epoch 23: val_accuracy did not improve from 0.82061
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9099 - loss: 0.3157 - val_accuracy: 0.8117 - val_loss: 0.5827 - learning_rate: 2.5000e-04
Epoch 24/30
195/197 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9152 - loss: 0.3096
Epoch 24: val_accuracy did not improve from 0.82061

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
197/197 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9152 - loss: 0.3097 - val_accuracy: 0.8092 - val_loss: 0.5858 - learning_rate: 2.5000e-04
Epoch 24: early stopping
Restoring model weights from the end of the best epoch: 20.
Epoch 1/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6245 - loss: 1.4256
Epoch 1: val_accuracy improved from -inf to 0.69466, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 28s 79ms/step - accuracy: 0.6247 - loss: 1.4244 - val_accuracy: 0.6947 - val_loss: 1.1210 - learning_rate: 1.0000e-05
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7831 - loss: 0.6756
Epoch 2: val_accuracy improved from 0.69466 to 0.76463, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.7831 - loss: 0.6755 - val_accuracy: 0.7646 - val_loss: 0.7904 - learning_rate: 1.0000e-05
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8249 - loss: 0.5242
Epoch 3: val_accuracy improved from 0.76463 to 0.80789, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.8249 - loss: 0.5240 - val_accuracy: 0.8079 - val_loss: 0.6306 - learning_rate: 1.0000e-05
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8527 - loss: 0.4374
Epoch 4: val_accuracy improved from 0.80789 to 0.81425, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.8527 - loss: 0.4373 - val_accuracy: 0.8142 - val_loss: 0.5785 - learning_rate: 1.0000e-05
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8824 - loss: 0.3357
Epoch 5: val_accuracy improved from 0.81425 to 0.83206, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.8824 - loss: 0.3357 - val_accuracy: 0.8321 - val_loss: 0.5253 - learning_rate: 1.0000e-05
Epoch 6/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8863 - loss: 0.3275
Epoch 6: val_accuracy improved from 0.83206 to 0.83461, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.8863 - loss: 0.3274 - val_accuracy: 0.8346 - val_loss: 0.5096 - learning_rate: 1.0000e-05
Epoch 7/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9061 - loss: 0.2869
Epoch 7: val_accuracy improved from 0.83461 to 0.83969, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9061 - loss: 0.2869 - val_accuracy: 0.8397 - val_loss: 0.4851 - learning_rate: 1.0000e-05
Epoch 8/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9176 - loss: 0.2402
Epoch 8: val_accuracy improved from 0.83969 to 0.84987, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9176 - loss: 0.2402 - val_accuracy: 0.8499 - val_loss: 0.4604 - learning_rate: 1.0000e-05
Epoch 9/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9299 - loss: 0.2069
Epoch 9: val_accuracy improved from 0.84987 to 0.85369, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9299 - loss: 0.2069 - val_accuracy: 0.8537 - val_loss: 0.4388 - learning_rate: 1.0000e-05
Epoch 10/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9323 - loss: 0.1973
Epoch 10: val_accuracy improved from 0.85369 to 0.85751, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9323 - loss: 0.1973 - val_accuracy: 0.8575 - val_loss: 0.4210 - learning_rate: 1.0000e-05
Epoch 11/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9419 - loss: 0.1736
Epoch 11: val_accuracy improved from 0.85751 to 0.86387, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9419 - loss: 0.1736 - val_accuracy: 0.8639 - val_loss: 0.4139 - learning_rate: 1.0000e-05
Epoch 12/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9429 - loss: 0.1645
Epoch 12: val_accuracy improved from 0.86387 to 0.86641, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9429 - loss: 0.1645 - val_accuracy: 0.8664 - val_loss: 0.4085 - learning_rate: 1.0000e-05
Epoch 13/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9462 - loss: 0.1517
Epoch 13: val_accuracy improved from 0.86641 to 0.87659, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9462 - loss: 0.1516 - val_accuracy: 0.8766 - val_loss: 0.3916 - learning_rate: 1.0000e-05
Epoch 14/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9610 - loss: 0.1222
Epoch 14: val_accuracy did not improve from 0.87659
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9610 - loss: 0.1222 - val_accuracy: 0.8753 - val_loss: 0.3802 - learning_rate: 1.0000e-05
Epoch 15/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9599 - loss: 0.1169
Epoch 15: val_accuracy improved from 0.87659 to 0.88295, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9599 - loss: 0.1169 - val_accuracy: 0.8830 - val_loss: 0.3744 - learning_rate: 1.0000e-05
Epoch 16/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9628 - loss: 0.1143
Epoch 16: val_accuracy did not improve from 0.88295
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9628 - loss: 0.1142 - val_accuracy: 0.8817 - val_loss: 0.3703 - learning_rate: 1.0000e-05
Epoch 17/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9689 - loss: 0.0996
Epoch 17: val_accuracy did not improve from 0.88295
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9689 - loss: 0.0996 - val_accuracy: 0.8830 - val_loss: 0.3629 - learning_rate: 1.0000e-05
Epoch 18/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9686 - loss: 0.0924
Epoch 18: val_accuracy did not improve from 0.88295
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9686 - loss: 0.0924 - val_accuracy: 0.8804 - val_loss: 0.3621 - learning_rate: 1.0000e-05
Epoch 19/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9761 - loss: 0.0828
Epoch 19: val_accuracy did not improve from 0.88295
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9761 - loss: 0.0828 - val_accuracy: 0.8804 - val_loss: 0.3533 - learning_rate: 1.0000e-05
Epoch 20/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9740 - loss: 0.0815
Epoch 20: val_accuracy improved from 0.88295 to 0.88931, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9740 - loss: 0.0815 - val_accuracy: 0.8893 - val_loss: 0.3463 - learning_rate: 1.0000e-05
Epoch 21/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9762 - loss: 0.0690
Epoch 21: val_accuracy improved from 0.88931 to 0.89186, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9762 - loss: 0.0690 - val_accuracy: 0.8919 - val_loss: 0.3453 - learning_rate: 1.0000e-05
Epoch 22/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9834 - loss: 0.0561
Epoch 22: val_accuracy improved from 0.89186 to 0.89313, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9834 - loss: 0.0561 - val_accuracy: 0.8931 - val_loss: 0.3350 - learning_rate: 1.0000e-05
Epoch 23/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9828 - loss: 0.0621
Epoch 23: val_accuracy did not improve from 0.89313
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9828 - loss: 0.0621 - val_accuracy: 0.8931 - val_loss: 0.3334 - learning_rate: 1.0000e-05
Epoch 24/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9836 - loss: 0.0616
Epoch 24: val_accuracy did not improve from 0.89313
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9836 - loss: 0.0616 - val_accuracy: 0.8919 - val_loss: 0.3293 - learning_rate: 1.0000e-05
Epoch 25/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9856 - loss: 0.0524
Epoch 25: val_accuracy improved from 0.89313 to 0.89440, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9855 - loss: 0.0525 - val_accuracy: 0.8944 - val_loss: 0.3289 - learning_rate: 1.0000e-05
Epoch 26/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9864 - loss: 0.0505
Epoch 26: val_accuracy did not improve from 0.89440
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9864 - loss: 0.0505 - val_accuracy: 0.8944 - val_loss: 0.3250 - learning_rate: 1.0000e-05
Epoch 27/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9872 - loss: 0.0452
Epoch 27: val_accuracy did not improve from 0.89440
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9872 - loss: 0.0452 - val_accuracy: 0.8931 - val_loss: 0.3246 - learning_rate: 1.0000e-05
Epoch 28/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9927 - loss: 0.0401
Epoch 28: val_accuracy improved from 0.89440 to 0.90331, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9927 - loss: 0.0401 - val_accuracy: 0.9033 - val_loss: 0.3167 - learning_rate: 1.0000e-05
Epoch 29/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9874 - loss: 0.0397
Epoch 29: val_accuracy did not improve from 0.90331
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9874 - loss: 0.0397 - val_accuracy: 0.9008 - val_loss: 0.3179 - learning_rate: 1.0000e-05
Epoch 30/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9865 - loss: 0.0447
Epoch 30: val_accuracy improved from 0.90331 to 0.90458, saving model to /kaggle/working/best_finetuned.h5
197/197 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9865 - loss: 0.0446 - val_accuracy: 0.9046 - val_loss: 0.3128 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 30.
Saved model to: /kaggle/working/mobilenet_finetuned.h5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8846 - loss: 0.3854
Test loss: 0.3207  Test accuracy: 0.8982
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step

Classification report (per-class):
              precision    recall  f1-score   support

           0     0.9000    1.0000    0.9474        27
           1     0.8571    1.0000    0.9231        18
           2     1.0000    1.0000    1.0000        29
           3     0.8378    0.9394    0.8857        33
           4     0.9231    0.8889    0.9057        27
           5     0.7200    0.9000    0.8000        20
           6     0.8571    0.9000    0.8780        20
           7     0.9524    1.0000    0.9756        20
           8     0.9333    0.7778    0.8485        18
           9     0.8519    0.9583    0.9020        24
          10     0.9565    0.9167    0.9362        24
          11     0.8788    0.9355    0.9062        31
          12     0.9615    0.8929    0.9259        28
          13     0.8800    0.9565    0.9167        23
          14     1.0000    0.7407    0.8511        27
          15     0.8636    0.7600    0.8085        25
          16     0.9118    0.9118    0.9118        34
          17     0.9412    0.7273    0.8205        22
          18     0.9444    0.7727    0.8500        22
          19     0.9032    0.9333    0.9180        30
          20     0.9500    0.9500    0.9500        20
          21     1.0000    0.9697    0.9846        33
          22     0.8824    0.8824    0.8824        17
          23     0.9231    0.8571    0.8889        42
          24     0.9545    0.9545    0.9545        22
          25     0.6538    0.8947    0.7556        19
          26     0.8519    0.7931    0.8214        29
          27     1.0000    1.0000    1.0000        24
          28     0.8000    0.9333    0.8615        30
          29     0.8696    0.9091    0.8889        22
          30     1.0000    0.7692    0.8696        26

    accuracy                         0.8982       786
   macro avg     0.9019    0.8976    0.8957       786
weighted avg     0.9055    0.8982    0.8981       786

Confusion matrix shape: (31, 31)

In [13]:
model.save("/kaggle/working/mobilenet_finetuned.h5")
